Minimal Spanning Tree, Prim's Algorithm, Kruskal's Algorithm

In [2]:
from Graph import *

In [4]:
G = Graph()

edge_list = [
    ('A', 'B'),
    ('A', 'D'),
    ('B', 'C'),
    ('B', 'D'),
    ('B', 'E'),
    ('C', 'E'),
    ('D', 'E'),
    ('D', 'F'),
    ('E', 'F'),
    ('E', 'G'),
    ('F', 'G')]

G.add_edges_from(edge_list)

G._edges['A']['B']['weight'] = 7
G._edges['A']['D']['weight'] = 5
G._edges['B']['C']['weight'] = 8
G._edges['B']['D']['weight'] = 9
G._edges['B']['E']['weight'] = 7
G._edges['C']['E']['weight'] = 5
G._edges['D']['E']['weight'] = 15
G._edges['D']['F']['weight'] = 6
G._edges['E']['F']['weight'] = 8
G._edges['E']['G']['weight'] = 9
G._edges['F']['G']['weight'] = 11

G.weights()

{('A', 'B'): 7,
 ('A', 'D'): 5,
 ('B', 'A'): 7,
 ('B', 'C'): 8,
 ('B', 'D'): 9,
 ('B', 'E'): 7,
 ('D', 'A'): 5,
 ('D', 'B'): 9,
 ('D', 'E'): 15,
 ('D', 'F'): 6,
 ('C', 'B'): 8,
 ('C', 'E'): 5,
 ('E', 'B'): 7,
 ('E', 'C'): 5,
 ('E', 'D'): 15,
 ('E', 'F'): 8,
 ('E', 'G'): 9,
 ('F', 'D'): 6,
 ('F', 'E'): 8,
 ('F', 'G'): 11,
 ('G', 'E'): 9,
 ('G', 'F'): 11}

# Prim's algorithm

In [5]:
G._nodes

{'A': {}, 'B': {}, 'D': {}, 'C': {}, 'E': {}, 'F': {}, 'G': {}}

In [6]:
@add_method(Graph)  # is a Jupyter notebook magic function, now we can call this method on a Graph object
# i.e. G.mst_prim(), even though it is not defined in the Graph class originally
def mst_prim(self):
    """ Calculates the minimum spanning tree using Prim's algorithm.
    Focus is to be human-readable; the code was not optimized for efficiency.
    """
    
    demo = True # Set this to true to print how the algorithm works step by step
    
    # initialize the MST nodes, links, and weights
    MST_nodes = set()
    MST_links = {} # Will look like this: {('A', 'B'): 5}

    # select an arbitrary node to begin with
    MST_nodes.add(list(self._nodes.keys())[0])
    
    while len(MST_nodes) != self.number_of_nodes():
        frontierlinks = set() # This is the frontier: All link candidates to grow the MST
        
        # for each node m in MST_nodes, consider the link (m, k) if
        # k is not already in MST_links
        for m in MST_nodes:
            for k in self._nodes.keys():
                if k not in MST_nodes and (m, k) in self.weights(): 
                    frontierlinks.add((m, k))
        
        # find the candidate link with the smallest weight
        newlink = sorted(frontierlinks, key=lambda e:self.weights()[e[0], e[1]])[0]
        # add this link and its weight to MST_links
        MST_links[newlink] = self.weights()[newlink[0], newlink[1]]
        # add the new node to MST_nodes
        MST_nodes.add(newlink[1])
        if demo: 
            print("Frontier:",frontierlinks)
            print("Adding the link",newlink,":",self.weights()[newlink[0], newlink[1]])
            print("MST progress:",MST_links, '\n')
        #if demo: print(newlink, self.weights()[newlink[0], newlink[1]])
        
    MST = Graph()
    MST.add_nodes_from(MST_nodes)
    MST.add_edges_from(MST_links.keys())
    for link, w in MST_links.items(): # update weights
        MST._edges[link[0]][link[1]]['weight'] = w
    
    return MST

In [7]:
MST = G.mst_prim()
MST.weights()

Frontier: {('A', 'B'), ('A', 'D')}
Adding the link ('A', 'D') : 5
MST progress: {('A', 'D'): 5} 

Frontier: {('D', 'E'), ('D', 'F'), ('A', 'B'), ('D', 'B')}
Adding the link ('D', 'F') : 6
MST progress: {('A', 'D'): 5, ('D', 'F'): 6} 

Frontier: {('D', 'B'), ('D', 'E'), ('F', 'G'), ('A', 'B'), ('F', 'E')}
Adding the link ('A', 'B') : 7
MST progress: {('A', 'D'): 5, ('D', 'F'): 6, ('A', 'B'): 7} 

Frontier: {('B', 'C'), ('D', 'E'), ('B', 'E'), ('F', 'G'), ('F', 'E')}
Adding the link ('B', 'E') : 7
MST progress: {('A', 'D'): 5, ('D', 'F'): 6, ('A', 'B'): 7, ('B', 'E'): 7} 

Frontier: {('B', 'C'), ('F', 'G'), ('E', 'C'), ('E', 'G')}
Adding the link ('E', 'C') : 5
MST progress: {('A', 'D'): 5, ('D', 'F'): 6, ('A', 'B'): 7, ('B', 'E'): 7, ('E', 'C'): 5} 

Frontier: {('F', 'G'), ('E', 'G')}
Adding the link ('E', 'G') : 9
MST progress: {('A', 'D'): 5, ('D', 'F'): 6, ('A', 'B'): 7, ('B', 'E'): 7, ('E', 'C'): 5, ('E', 'G'): 9} 



{('A', 'D'): 5,
 ('A', 'B'): 7,
 ('D', 'A'): 5,
 ('D', 'F'): 6,
 ('F', 'D'): 6,
 ('B', 'A'): 7,
 ('B', 'E'): 7,
 ('E', 'B'): 7,
 ('E', 'C'): 5,
 ('E', 'G'): 9,
 ('C', 'E'): 5,
 ('G', 'E'): 9}

# Kruskal's algorithm

In [2]:
@add_method(Graph)
class _UF:
    # Union-Find data structure, used for Kruskal's algorithm, important
    """Union-find data structure for the Kruskal algorithm.
    Focus is to be human-readable; the code was not optimized for efficiency.
    """

    def __init__(self, G):
        """Initialize: Every node is its own parent and has rank 0"""
        self.parent = {key:key for key in list(G._nodes.keys())}
        self.rank = {key:0 for key in list(G._nodes.keys())}

    def are_connected(self, u, v):
        """Check if two nodes are in the same component"""
        return self._find(u) == self._find(v)

    def _find(self, u):
        """Find the root of node u"""
        while u != self.parent[u]:
            u = self.parent[u]
        return u
    
    def union(self, u, v):
        """Create the union of two components"""
        u_root = self._find(u)
        v_root = self._find(v)
        if u_root == v_root:
            # nothing to do if they are already in the same component
            return
        if self.rank[u_root] > self.rank[v_root]:
            # If u is higher ranked, we can connect v to u
            self.parent[v_root] = u_root
        else:
            # If not, we connect u to v and increase v's rank if needed
            self.parent[u_root] = v_root
            if self.rank[u_root] == self.rank[v_root]:
                self.rank[v_root] += 1

NameError: name 'add_method' is not defined

In [1]:
@add_method(Graph)
def mst_kruskal(self):
    """ Calculates the minimum spanning tree using Kruskal's algorithm.
    Focus is to be human-readable; the code was not optimized for efficiency.
    """
    
    demo = True # Set this to true to print how the algorithm works step by step
    
    # Initialize the MST nodes, links, and weights
    MST_nodes = set()
    MST_links = {} # Will look like this: {('A', 'B'): 5}

    # Sort all links by weights from smallest to largest. Skip every other due to symmetry.
    sorted_links = sorted(self.weights(), key=self.weights().__getitem__)[0::2]
    
    uf = _UF(self)
    for e in sorted_links:
        u, v = e
        if demo: print("Consider adding:",e,":",self.weights()[e])
        # if u, v already connected, abort this edge
        if uf.are_connected(u, v):
            if demo: print("NOT adding the link.\n")
            continue
        # if not, connect them and add this edge and its nodes to the MST
        uf.union(u, v)
        MST_links[e] = self.weights()[e[0], e[1]]
        MST_nodes.add(u)
        MST_nodes.add(v)
        if demo: 
            print("Adding the link.")
            print("MST progress:",MST_links)
            print("Components:", uf.parent)
            print("Ranks:", uf.rank, "\n")
        if len(MST_nodes) == self.number_of_nodes(): 
            break # We can stop once we cover all nodes
        
    MST = Graph()
    MST.add_nodes_from(MST_nodes)
    MST.add_edges_from(MST_links.keys())
    for link, w in MST_links.items(): # update weights
        MST._edges[link[0]][link[1]]['weight'] = w
    
    return MST

NameError: name 'add_method' is not defined

In [10]:
MST = G.mst_kruskal()
MST.weights()

Consider adding: ('A', 'D') : 5
Adding the link.
MST progress: {('A', 'D'): 5}
Components: {'A': 'D', 'B': 'B', 'D': 'D', 'C': 'C', 'E': 'E', 'F': 'F', 'G': 'G'}
Ranks: {'A': 0, 'B': 0, 'D': 1, 'C': 0, 'E': 0, 'F': 0, 'G': 0} 

Consider adding: ('C', 'E') : 5
Adding the link.
MST progress: {('A', 'D'): 5, ('C', 'E'): 5}
Components: {'A': 'D', 'B': 'B', 'D': 'D', 'C': 'E', 'E': 'E', 'F': 'F', 'G': 'G'}
Ranks: {'A': 0, 'B': 0, 'D': 1, 'C': 0, 'E': 1, 'F': 0, 'G': 0} 

Consider adding: ('D', 'F') : 6
Adding the link.
MST progress: {('A', 'D'): 5, ('C', 'E'): 5, ('D', 'F'): 6}
Components: {'A': 'D', 'B': 'B', 'D': 'D', 'C': 'E', 'E': 'E', 'F': 'D', 'G': 'G'}
Ranks: {'A': 0, 'B': 0, 'D': 1, 'C': 0, 'E': 1, 'F': 0, 'G': 0} 

Consider adding: ('A', 'B') : 7
Adding the link.
MST progress: {('A', 'D'): 5, ('C', 'E'): 5, ('D', 'F'): 6, ('A', 'B'): 7}
Components: {'A': 'D', 'B': 'D', 'D': 'D', 'C': 'E', 'E': 'E', 'F': 'D', 'G': 'G'}
Ranks: {'A': 0, 'B': 0, 'D': 1, 'C': 0, 'E': 1, 'F': 0, 'G': 0} 

{('A', 'D'): 5,
 ('A', 'B'): 7,
 ('D', 'A'): 5,
 ('D', 'F'): 6,
 ('C', 'E'): 5,
 ('E', 'C'): 5,
 ('E', 'B'): 7,
 ('E', 'G'): 9,
 ('F', 'D'): 6,
 ('B', 'A'): 7,
 ('B', 'E'): 7,
 ('G', 'E'): 9}